#### Урок 3. Парсинг HTML. BS, SQLAlchemy

1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД
2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы
3*)Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [ ]:
# Установка БД
"""
docker run -d -p 127.0.0.1:27017:27017 --name mongo4parsing mongo
docker ps -a
docker start f48a0957b452
docker exec -it f48a0957b452 mongo
"""

In [13]:
from pymongo import MongoClient
#from pprint import pprint
import pandas as pd
import numpy as np


client = MongoClient('mongodb://127.0.0.1:27017')
vacancy_pd = pd.read_csv('vacancy_pd.csv')

In [22]:
vacancy_pd.head()

,Unnamed: 0,employer,link,max_salary,min_salary,vacancy,vacancy_link
0,0,"[' Ренессанс cтрахование, Группа']",['https://hh.ru'],[nan],[nan],['Методолог/Руководитель направления методолог...,['https://hh.ru/vacancy/34005950?query=%D0%BC%...
1,1,[' Всероссийский Банк Развития Регионов (ВБРР)'],['https://hh.ru'],[nan],[nan],['Методолог кадрового администрирования'],['https://hh.ru/vacancy/34240183?query=%D0%BC%...
2,2,"[' Ренессанс cтрахование, Группа']",['https://hh.ru'],[nan],[nan],['Ведущий специалист/Риск-менеджер в Управлени...,['https://hh.ru/vacancy/34195483?query=%D0%BC%...
3,3,['ООО EveryCo'],['https://hh.ru'],['150000'],[nan],['Методолог'],['https://hh.ru/vacancy/34180881?query=%D0%BC%...
4,4,['ООО Верме'],['https://hh.ru'],[nan],[nan],['Методолог'],['https://hh.ru/vacancy/33999225?query=%D0%BC%...


In [ ]:
def db_vacancy_writer(df):
    vacancy_db = client['vacancy']
    docs = vacancy_db.docs
    result = docs.insert_many(df.to_dict(orient='records'))
    return result.inserted_ids

In [19]:
db_vacancy_writer(vacancy_pd)

In [43]:
def db_vacancy_search(salary):
    vacancy_db = client['vacancy']
    docs = vacancy_db.docs
    for x in docs.find({},{ "min_salary": {'$gt':salary} }):
        print(docs['vacancy'])

In [44]:
db_vacancy_search(100000)

OperationFailure: Unsupported projection option: min_salary: { $gt: "salary" }

In [5]:
db = client['test']
docs = db.docs
doc_data = {
     "name": "One more document",
     "author": {
             "fullname": "Sergie",
             "age": 18,
             "address": "Алтуфьево"
     },
     "created": "01.12.1982",
     "genres": ["философия", "action", "psy"]
 }
result  = docs.insert_one(doc_data)
print(f"New doc: {result.inserted_id}")


New doc: 5db20e402f49ddca3916cfa1


In [10]:
phil_doc = db.docs.find()
print(phil_doc)
for doc in phil_doc:
    print(doc)

{'_id': ObjectId('5db20e402f49ddca3916cfa1'), 'name': 'One more document', 'author': {'fullname': 'Sergie', 'age': 18, 'address': 'Алтуфьево'}, 'created': '01.12.1982', 'genres': ['философия', 'action', 'psy']}
{'_id': ObjectId('5db20e8f7084bedcf3838e33'), 'name': 'test doc', 'size': 1000.0, 'author': 'Konan Varvar'}
{'_id': ObjectId('5db20e937084bedcf3838e34'), 'name': 'test doc', 'size': 1000.0, 'author': 'Konan Varvar'}
{'_id': ObjectId('5db20edb7084bedcf3838e35'), 'name': 'Very Important Document', 'author': {'fullname': 'Ivanov Ivan', 'age': 78.0, 'address': 'LA, St. Patrick street'}, 'created': '19.03.1990', 'genres': ['thriller', 'action', 'psy']}


In [ ]:
db = client['test']
docs = db.docs
doc_data = {
     "name": "One more document",
     "author": {
             "fullname": "Sergie",
             "age": 18,
             "address": "Алтуфьево"
     },
     "created": "01.12.1982",
     "genres": ["философия", "action", "psy"]
 }
result  = docs.insert_one(doc_data)
print(f"New doc: {result.inserted_id}")

New doc: 5b4f1c841987fddee07a96e2
>>> phil_doc = docs.find_one({"genres": "философия"})
>>> print(phil_doc)
{'_id': ObjectId('5b4f1c841987fddee07a96e2'), 'name': 'One more document', 'author': {'fullname': 'Sergie', 'age': 18, 'address': 'Алтуфьево'}, 'created': '01.12.1982', 'genres': ['философия', 'action', 'psy']}


In [ ]:
>>> from pymongo import MongoClient
>>> db = MongoClient('mongodb://127.0.0.1:27017').aggr_test
>>> result = db.animals.insert_many([{"x": 1, "tags": ["dog", "cat"]}, {"x": 2, "tags": ["cat"]}, {"x": 2, "tags": ["mouse", "cat", "dog"]},{"x": 3, "tags": []}])
>>> result.inserted_ids
[ObjectId('5b4f48dd1987fde7d281d065'), ObjectId('5b4f48dd1987fde7d281d066'), ObjectId('5b4f48dd1987fde7d281d067'), ObjectId('5b4f48dd1987fde7d281d068')]


In [ ]:
import pandas as pd
from pymongo import MongoClient


def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

In [ ]:
client = MongoClient()
db = client.database_name
collection = db.collection_name
data = pd.DataFrame(list(collection.find()))